# Data Integrity
In this notebook, we do several sanity checks on our data.
If the sanity checks are not fulfilled either something went wrong,
or our assumptions were wrong.

# Ghidra Analysis Failures

Q: How many binaries could not be analyzed?

A: 75 out of 66,580

```sql 
SELECT *
FROM "binary" b
	LEFT OUTER JOIN v.executable2binary e2b ON (
		b.id = e2b.binary_id
	)
WHERE e2b.binary_id IS NULL
```

Q: How many binaries were analyzed by ghidra that are not in the evaluatie database?
\
Note that this should not happen, as we only analyze the binaries that are in the database.

A: None.

```sql
SELECT *
FROM v.bsim_exetable e
	LEFT OUTER JOIN v.executable2binary e2b ON (
		e.id = e2b.executable_id
	)
WHERE e2b.executable_id IS NULL AND
	-- Ghidra adds dynamically linked libraries with a fake hash.
	e.md5 NOT LIKE 'bbbbbbbbaaaaaaaa%'
```

# Ghidra Description to Evaluatie Function Mapping

Q: How many functions that ghidra found were not part of the evaluatie dataset when mapping by address?

A: 1,387,404 out of 20,725,781. 

```sql
SELECT count(*)
FROM v.bsim_desctable d LEFT OUTER JOIN v.description2function d2f ON (
	d.id = d2f.description_id
)
WHERE d2f.description_id IS NULL
``` 

Q: Which binaries have unmapped descriptions that are named?\
Note that unnamed functions (i.e. starting with 'FUN_') are arguably not that relevant for us
as they are not part of the debug symbols (and we cannot know their labels anyways).


A: 5,878 out of 66,580.

```sql
WITH description_wo_function AS (
	SELECT d.id, d.name_func AS name, d.id_exe AS executable_id
	FROM v.bsim_desctable d LEFT OUTER JOIN v.description2function d2f ON (
		d.id = d2f.description_id
	)
	WHERE d2f.description_id IS NULL
)
SELECT DISTINCT e2b.binary_id
FROM description_wo_function d JOIN v.executable2binary e2b ON (
	d.executable_id = e2b.executable_id
) JOIN "binary" b ON (
	b.id = e2b.binary_id
)
WHERE d.name NOT LIKE 'FUN_%' 
```

Q: Which binaries have unmapped functions?


A: All of them.

```sql
WITH function_wo_description AS (
	SELECT f.*
	FROM "function" f
		LEFT OUTER JOIN v.description2function d2f ON (
			f.id = d2f.function_id
		)
	WHERE d2f.function_id IS NULL
)
SELECT DISTINCT f.binary_id
FROM function_wo_description f
	JOIN "binary" b ON (
		f.binary_id = b.id
	)
```

Q: Can we correct the binaries with no base address?

A: Yes! Actually, this is pretty simple as the base address is one of two values.
One value for 32 bit systems and another value for 64 bit systems.

```sql
WITH "binary:wired" AS (
	SELECT b.*, ARRAY_AGG(DISTINCT d.addr - f.offset) AS bases
	FROM "binary" b
		JOIN v.executable2binary e2b ON (
			b.id = e2b.binary_id
		)
		JOIN v.bsim_desctable d ON (
			d.id_exe = e2b.executable_id
		)
		JOIN "function" f ON (
			b.id = f.binary_id AND
			d.name_func = f.name
		)
		JOIN build_parameters bp ON (
			b.build_parameters_id = bp.id
		)
	WHERE b.image_base = 0
	GROUP BY b.id
), "binary:new-base" AS (
	SELECT
		b.*,
		(
		CASE WHEN 65536 = ANY(b.bases) AND NOT (1048576 = ANY(b.bases)) THEN 65536
			 WHEN 1048576 = ANY(b.bases) AND NOT (65536 = ANY(b.bases)) THEN 1048576
			 ELSE NULL
		END
		) AS new_base
	FROM "binary:wired" b

)
SELECT DISTINCT b.new_base, bp.bitness
FROM "binary:new-base" b
	JOIN build_parameters bp ON (
		b.build_parameters_id = bp.id
	)

```

Q: Do image base and bitness correspond one to one?

A: No.

```sql
SELECT b.image_base, bp.bitness, COUNT(b.id)
FROM "binary" b
	JOIN build_parameters bp ON (
		b.build_parameters_id = bp.id
	)
GROUP BY b.image_base, bp.bitness
```


## Ghidra Function to IDA Function Mapping
We use Ghidra for decompilation but our ground truth set uses IDA.
In this section, we validate that our database looks as if we never used IDA, but only Ghidra.


```sql
-- Which functions have different names in Ghidra/IDA.

WITH binary_analyzed AS
	(SELECT b.id as binary_id, e.id as executable_id
	FROM "binary" AS b
	JOIN bsim.exetable AS e
	ON b.md5 = e.md5),
function_data AS (
	SELECT f.id AS id, f.offset + b.image_base AS address, b.id AS binary_id, f.name
	FROM "function" AS f JOIN "binary" AS b ON f.binary_id = b.id
)
	
SELECT *
FROM bsim.desctable AS description
	-- Join all analyzed binaries to discard linked functions
	JOIN binary_analyzed ON (
		description.id_exe = binary_analyzed.executable_id
	)
	-- Outer join to not discard functions that ghidra found
	LEFT OUTER JOIN function_data ON (
		binary_analyzed.binary_id = function_data.binary_id AND
		description.addr = function_data.address
	)
WHERE NOT (
	function_data.name ~ ('^' || description.name_func || '(_[0-9]+)?$')
)
 ```

In [1]:
SELECT *
FROM v.description2function d2f
	JOIN "function" f ON (
		d2f.function_id = f.id
	)
	JOIN v.bsim_desctable d ON (
		d2f.description_id = d.id
	)

SyntaxError: invalid syntax (2250411566.py, line 1)

```sql
-- Which functions did Ghidra find, that IDA did not find?

	
SELECT *
FROM bsim.desctable AS description
	-- Join all analyzed binaries to discard linked functions
	JOIN binary_analyzed ON (
		description.id_exe = binary_analyzed.executable_id
	)
	-- Outer join to not discard functions that ghidra found
	LEFT OUTER JOIN function_data ON (
		binary_analyzed.binary_id = function_data.binary_id AND
		description.addr = function_data.address
		-- In Ghidra, multiple functions (as defined by their address) can have the same name.
		-- In IDA, a counter is appended.
		AND function_data.name ~ ('^' || description.name_func || '(_[0-9]+)?$')
	)
--WHERE function_data.id IS NULL
```



```sql
-- How many intra-binary edges are in Ghidra's call-graph?

WITH dynlibs AS (
	SELECT *
	FROM bsim.exetable
	WHERE name_exec = 'unknown'
)
SELECT count(*)
FROM bsim.callgraphtable AS cg
	JOIN bsim.desctable AS src ON (
		cg.src = src.id
	)
	JOIN bsim.desctable AS dst ON (
		cg.dest = dst.id
	)
WHERE NOT EXISTS (SELECT * FROM dynlibs WHERE src.id_exe = dynlibs.id) AND
NOT EXISTS (SELECT * FROM dynlibs WHERE dst.id_exe = dynlibs.id)

```

```sql
-- Does it match the amount of edges in our call-graph?
-- Turns out we loose very vew.

SELECT count(*)
FROM call_graph_edge
```